In [1]:
sc

StatementMeta(sparkpool, 23, 2, Finished, Available, Finished)

<SparkContext master=yarn appName=taxianalysis_final_sparkpool_1721380014>

In [2]:
spark

StatementMeta(sparkpool, 23, 3, Finished, Available, Finished)

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

StatementMeta(sparkpool, 23, 4, Finished, Available, Finished)

In [4]:
trip_df = spark.read.load("abfss://datasynapsefs@datasynapselake.dfs.core.windows.net/TripData.csv", header =True, inferschema = True, format = "csv")


StatementMeta(sparkpool, 23, 5, Finished, Available, Finished)

In [5]:
trip_df.printSchema()

StatementMeta(sparkpool, 23, 6, Finished, Available, Finished)

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [6]:
print(trip_df.columns)

StatementMeta(sparkpool, 23, 7, Finished, Available, Finished)

['VendorID', 'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'total_amount', 'payment_type', 'trip_type', 'congestion_surcharge']


In [7]:
#filter records with no passenger count and '0' trip distance
trip_df1 = trip_df.where(col('passenger_count')>0).filter(col('trip_distance')>0.0).drop_duplicates() 


StatementMeta(sparkpool, 23, 8, Finished, Available, Finished)

In [8]:
print(trip_df1.rdd.getNumPartitions())

StatementMeta(sparkpool, 23, 9, Finished, Available, Finished)

4


In [9]:
trip_df2 = trip_df1.select(col('VendorID'),
col('lpep_pickup_datetime').alias('PickUpTime'),
col('lpep_dropoff_datetime').alias('DropTime'),
col('passenger_count').alias('PassengerCount'),
col('trip_distance').alias('TripDistance'),
col('PULocationID').alias('PickUpLocation'),
col('DOLocationID').alias('DropLocationID'),col('RatecodeID'),
col('fare_amount'),col('total_amount'),col('payment_type'),col('trip_type'),
)

StatementMeta(sparkpool, 23, 10, Finished, Available, Finished)

In [10]:
trip_df2.show(1)

StatementMeta(sparkpool, 23, 11, Finished, Available, Finished)

+--------+--------------------+--------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+
|VendorID|          PickUpTime|            DropTime|PassengerCount|TripDistance|PickUpLocation|DropLocationID|RatecodeID|fare_amount|total_amount|payment_type|trip_type|
+--------+--------------------+--------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+
|       2|01/01/2021 08:59:...|01/01/2021 09:04:...|             1|        1.44|            74|            75|         1|        6.5|        8.76|           1|        1|
+--------+--------------------+--------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+
only showing top 1 row



In [11]:
trip_df2.select('PickUpTime').show(2,truncate=False)

StatementMeta(sparkpool, 23, 12, Finished, Available, Finished)

+----------------------+
|PickUpTime            |
+----------------------+
|01/01/2021 08:59:00 AM|
|01/01/2021 11:56:17 AM|
+----------------------+
only showing top 2 rows



In [12]:
trip_df2.select('RatecodeID').distinct().show()

StatementMeta(sparkpool, 23, 13, Finished, Available, Finished)

+----------+
|RatecodeID|
+----------+
|         1|
|         3|
|         5|
|         4|
|         2|
|         6|
+----------+



In [13]:
from pyspark.sql.functions import to_timestamp

datetime_format = "MM/dd/yyyy hh:mm:ss a"

trip_df2 = trip_df2.withColumn('PickUpTime', to_timestamp('PickUpTime', datetime_format))
trip_df2 = trip_df2.withColumn('DropTime', to_timestamp('DropTime', datetime_format))


StatementMeta(sparkpool, 23, 14, Finished, Available, Finished)

In [14]:
trip_df3 = trip_df2.withColumn("TripYear",year("PickUpTime"))\
.withColumn("TripMonth",month("PickUpTime"))\
.withColumn("TripDay",dayofmonth("PickUpTime"))\
.withColumn("TripDuration",round((unix_timestamp(col("DropTime"))-unix_timestamp(col("PickUpTime")))/60,2))

StatementMeta(sparkpool, 23, 15, Finished, Available, Finished)

In [15]:
trip_df3.show(1)

StatementMeta(sparkpool, 23, 16, Finished, Available, Finished)

+--------+-------------------+-------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+--------+---------+-------+------------+
|VendorID|         PickUpTime|           DropTime|PassengerCount|TripDistance|PickUpLocation|DropLocationID|RatecodeID|fare_amount|total_amount|payment_type|trip_type|TripYear|TripMonth|TripDay|TripDuration|
+--------+-------------------+-------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+--------+---------+-------+------------+
|       2|2021-01-01 08:59:00|2021-01-01 09:04:13|             1|        1.44|            74|            75|         1|        6.5|        8.76|           1|        1|    2021|        1|      1|        5.22|
+--------+-------------------+-------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+-

In [16]:
trip_df4 = trip_df3.withColumn("TripMethod",when(col("RatecodeID")==6,"SharedTrip").otherwise("SoloTrip"))
trip_df4.show()

StatementMeta(sparkpool, 23, 17, Finished, Available, Finished)

+--------+-------------------+-------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+--------+---------+-------+------------+----------+
|VendorID|         PickUpTime|           DropTime|PassengerCount|TripDistance|PickUpLocation|DropLocationID|RatecodeID|fare_amount|total_amount|payment_type|trip_type|TripYear|TripMonth|TripDay|TripDuration|TripMethod|
+--------+-------------------+-------------------+--------------+------------+--------------+--------------+----------+-----------+------------+------------+---------+--------+---------+-------+------------+----------+
|       2|2021-01-01 08:59:00|2021-01-01 09:04:13|             1|        1.44|            74|            75|         1|        6.5|        8.76|           1|        1|    2021|        1|      1|        5.22|  SoloTrip|
|       1|2021-01-01 11:56:17|2021-01-01 11:58:47|             1|         0.8|            74|            75|         1|     

In [17]:
spark.sql("CREATE DATABASE IF NOT EXISTS GreenNYCTaxi")

StatementMeta(sparkpool, 23, 18, Finished, Available, Finished)

DataFrame[]

In [28]:
trip_df4.drop('RateCodeID').repartition(4).write.mode("overwrite").saveAsTable('GreenNYCTaxi.TaxiTable')

StatementMeta(sparkpool, 23, 29, Finished, Available, Finished)

In [19]:
#AvgTrip Duration
spark.sql("SELECT AVG(TripDuration) AS AverageTripDuration FROM TaxiTable").show()

StatementMeta(sparkpool, 23, 20, Finished, Available, Finished)

+-------------------+
|AverageTripDuration|
+-------------------+
| 23.971967633371385|
+-------------------+



In [20]:
#Distinct trip methods
spark.sql("SELECT TripMethod, COUNT(*) AS TripCount FROM TaxiTable GROUP BY TripMethod;").show()

StatementMeta(sparkpool, 23, 21, Finished, Available, Finished)

+----------+---------+
|TripMethod|TripCount|
+----------+---------+
|SharedTrip|        3|
|  SoloTrip|   621007|
+----------+---------+



In [21]:
%%sql
SELECT fare_amount, total_amount
FROM TaxiTable;


StatementMeta(sparkpool, 23, 22, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 2 fields>

In [22]:
%%sql
SELECT TripDistance, TripDuration FROM TaxiTable ORDER BY TripDuration DESC LIMIT 10 ;
-- top 5 longest trip

StatementMeta(sparkpool, 23, 23, Finished, Available, Finished)

<Spark SQL result set with 10 rows and 2 fields>

In [23]:
%%sql 
SELECT PickUpLocation, COUNT(*) AS PickupCount
FROM TaxiTable
GROUP BY PickUpLocation
ORDER BY PickupCount DESC
LIMIT 5 ;

-- top 5 pickup location

StatementMeta(sparkpool, 23, 24, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

In [24]:
%%sql
SELECT DropLocationID, COUNT(*) AS DropCount
FROM TaxiTable
GROUP BY DropLocationID
ORDER BY DropCount DESC
LIMIT 5;

-- top 5 drop location

StatementMeta(sparkpool, 23, 25, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

In [25]:
%%sql
SELECT payment_type, COUNT(*) AS PaymentCount
FROM TaxiTable
GROUP BY payment_type;

-- Payment Type vs Payment Count

StatementMeta(sparkpool, 23, 26, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>

In [26]:
%%sql
SELECT TripYear,TripMonth,Sum(PassengerCount) FROM TAXITABLE GROUP BY TripYear,TripMonth

StatementMeta(sparkpool, 23, 27, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 3 fields>